In [1]:
import numpy as np
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from dotenv import load_dotenv
from langchain_chroma import Chroma
from uuid import uuid4

load_dotenv()

True

In [2]:
csv_path = 'E:\JK Voice Assistant\dataset.csv'
data = pd.read_csv(csv_path,lineterminator='\n')

In [3]:
data.head()

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights,image_links\r
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,NaN,Cricket Practice Net NYLON HDPE Material W x H...,https://rukminim1.flixcart.com/image/612/612/x...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...,https://rukminim1.flixcart.com/image/612/612/x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,NaN,Cricket Rubber Ball Weight: 110 g,https://rukminim1.flixcart.com/image/612/612/x...
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,NaN,Cricket Synthetic Ball Weight: 110 g,https://rukminim1.flixcart.com/image/612/612/k...
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...,https://rukminim1.flixcart.com/image/612/612/j...


In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   category_1      12041 non-null  object 
 1   category_2      12041 non-null  object 
 2   category_3      12041 non-null  object 
 3   title           12023 non-null  object 
 4   product_rating  11961 non-null  float64
 5   selling_price   12013 non-null  object 
 6   mrp             11666 non-null  object 
 7   seller_name     11825 non-null  object 
 8   seller_rating   11827 non-null  float64
 9   description     5021 non-null   object 
 10  highlights      6560 non-null   object 
    12041 non-null  object 
dtypes: float64(2), object(10)
memory usage: 1.1+ MB


In [5]:
data.isna().sum()

category_1           0
category_2           0
category_3           0
title               18
product_rating      80
selling_price       28
mrp                375
seller_name        216
seller_rating      214
description       7020
highlights        5481
image_links\r        0
dtype: int64

In [6]:
data.drop(columns = ['image_links\r'], inplace=True)

In [7]:
data = data.dropna(subset = ['mrp'], how = 'all')

In [8]:
data.isna().sum()

category_1           0
category_2           0
category_3           0
title                0
product_rating      60
selling_price        0
mrp                  0
seller_name        183
seller_rating      181
description       6782
highlights        5301
dtype: int64

In [9]:
data['seller_rating'].fillna(value = 0, inplace=True)
data['product_rating'].fillna(value = 0, inplace=True)
data['seller_name'].fillna(value="", inplace=True)
data['description'].fillna(value="", inplace=True)
data['highlights'].fillna(value="", inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_36964\1760232250.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['seller_rating'].fillna(value = 0, inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_36964\1760232250.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [10]:
documents = []

for index, row in data.iterrows():
    description = Document(
        page_content = f"The category of this product is {row['category_1']} and subcategories are {row['category_2']} and {row['category_3']}. The title of the product is '{row['title']}' with a product rating of {row['product_rating']} and a selling price of {row['selling_price']} and MRP {row['mrp']}). The product is sold by {row['seller_name']} with a seller rating of {row['seller_rating']}. Product description: {row['description']}. Highlights include: {row['highlights']}.",
        metadata={"source":f"{row['category_1']}"},
    )
    documents.append(description)

In [11]:
# print(len(documents))
documents[0]

Document(page_content="The category of this product is Sports, Books and More and subcategories are Sports and Cricket . The title of the product is 'ITWOSERVICES CRICKET NET 100X10 CRICKET NET NYLON HDPE Cricket Net\xa0\xa0(Green)' with a product rating of 4.4 and a selling price of ₹1,615 and MRP ₹4,000). The product is sold by I2SERVICES with a seller rating of 4.4. Product description: . Highlights include: Cricket Practice Net NYLON HDPE Material W x H x D: 10 x 10.", metadata={'source': 'Sports, Books and More'})

In [12]:
temp_docs = documents[0:5]
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("inference_api_key"), model_name="sentence-transformers/all-MiniLM-l6-v2"
)

vector_store = Chroma(
    collection_name="product_collection",
    embedding_function=embeddings,
    persist_directory="E:\JK Voice Assistant\chromadb_langchain",  # Where to save data locally, remove if not neccesary,
    collection_metadata={"hnsw:space": "cosine"}
)

uuids = [str(uuid4()) for _ in range(len(temp_docs))]

vector_store.add_documents(documents=temp_docs)


['81ae7a1b-a78c-4aa0-a50f-a695753661bc',
 'a0912838-f067-4edd-817d-27a860e7624a',
 '0d732516-b895-4216-a9f4-7d6c82d8315d',
 'abfd0b4b-d7a7-447a-90f5-d7a80c631647',
 'fa063d77-53b3-4f1d-a6db-4619659d8101']

In [16]:
vector_store.get()

{'ids': ['0d732516-b895-4216-a9f4-7d6c82d8315d',
  '81ae7a1b-a78c-4aa0-a50f-a695753661bc',
  '90c85678-a806-458d-badd-6f4d7b2e439f',
  '9e783494-37d0-4663-acfe-6db53ed27d11',
  'a0912838-f067-4edd-817d-27a860e7624a',
  'abfd0b4b-d7a7-447a-90f5-d7a80c631647',
  'b57ea949-5af4-4513-9912-ec6e45d65d2b',
  'c84723ac-f9f8-4ffc-a23a-185e76d049a2',
  'e50acc0b-2d3c-415f-bad0-cb6a0b692b5d',
  'fa063d77-53b3-4f1d-a6db-4619659d8101'],
 'embeddings': None,
 'metadatas': [{'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'},
  {'source': 'Sports, Books and More'}],
 'documents': ["The category of this product is Sports, Books and More and subcategories are Sports and Cricket . The title of the produc

In [14]:
docs = []
ids=[]
for index, row in data.iterrows():
    text = f"The category of this product is {row['category_1']} and subcategories are {row['category_2']} and {row['category_3']}. The title of the product is '{row['title']}' with a product rating of {row['product_rating']} and a selling price of {row['selling_price']} and MRP {row['mrp']}). The product is sold by {row['seller_name']} with a seller rating of {row['seller_rating']}. Product description: {row['description']}. Highlights include: {row['highlights']}."
    docs.append(text)
    ids.append(str(index+1))

In [13]:
# results = vector_store.similarity_search_by_vector(
#     "I want cricket bat and net",
#     k=2,
# )
results = vector_store.similarity_search_by_vector(
    embedding=embeddings.embed_query("I want cricket bat and net"), k=2
)

In [14]:
results

[Document(page_content="The category of this product is Sports, Books and More and subcategories are Sports and Cricket . The title of the product is 'ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 10X10 FEET Cricket Net\xa0\xa0(Green)' with a product rating of 4.4 and a selling price of ₹152 and MRP ₹600). The product is sold by I2SERVICES with a seller rating of 4.4. Product description: 10 X  10 GREEN CRICKET NET HDPE NYLON.. Highlights include: Cricket HDPE NYLON Material W x H x D: 3.048 x 3.048.", metadata={'source': 'Sports, Books and More'}),
 Document(page_content="The category of this product is Sports, Books and More and subcategories are Sports and Cricket . The title of the product is 'ITWOSERVICES CRICKET NET 100X10 CRICKET NET NYLON HDPE Cricket Net\xa0\xa0(Green)' with a product rating of 4.4 and a selling price of ₹1,615 and MRP ₹4,000). The product is sold by I2SERVICES with a seller rating of 4.4. Product description: . Highlights include: Cricket Practice Net NYLON HD

In [28]:
data.head()

,category_1,category_2,category_3,title,product_rating,selling_price,mrp,seller_name,seller_rating,description,highlights
0,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET 100X10 CRICKET NET NY...,4.4,"₹1,615","₹4,000",I2SERVICES,4.4,,Cricket Practice Net NYLON HDPE Material W x H...
1,"Sports, Books and More",Sports,Cricket,ITWOSERVICES CRICKET NET GROUND BOUNDARY NET 1...,4.4,₹152,₹600,I2SERVICES,4.4,10 X 10 GREEN CRICKET NET HDPE NYLON.,Cricket HDPE NYLON Material W x H x D: 3.048 x...
2,"Sports, Books and More",Sports,Cricket,VICTORY Medium Weight ( Pack of 1 ) Rubber Cri...,3.7,₹59,₹199,VictoryOutlets,4.7,,Cricket Rubber Ball Weight: 110 g
3,"Sports, Books and More",Sports,Cricket,VICTORY Cricket Wind Ball (Pack of 1) - Made i...,3.8,₹75,₹299,VictoryOutlets,4.7,,Cricket Synthetic Ball Weight: 110 g
4,"Sports, Books and More",Sports,Cricket,CEAT Hitman Full Size Double Blade Poplar Cric...,3.4,₹329,"₹1,399",IndiaFit,4.7,The Ceat Poplar Willow Cricket Bat has been de...,Age Group 15+ Yrs Blade Made of Poplar Willow ...


In [27]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
            api_key=os.getenv("inference_api_key"), model_name="sentence-transformers/all-MiniLM-l6-v2"
        )

db = Chroma(persist_directory="E:\JK Voice Assistant\chromadb_langchain", embedding_function=embeddings)

In [28]:
db.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [29]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("inference_api_key"), model_name="sentence-transformers/all-MiniLM-l6-v2"
)
persist_directory="E:\JK Voice Assistant\chromadb_langchain"
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
retriever = vectordb.as_retriever()


In [37]:
print(retriever.name)

None


In [18]:
results = vector_store.similarity_search_with_score(
    "Tell me about a cat", k=2
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.860994] The category of this product is Sports, Books and More and subcategories are Sports and Cricket . The title of the product is 'CEAT Hitman Full Size Double Blade Poplar Cricket Bat - Advance Play Poplar Willow Cricket  Bat  (1.2 kg)' with a product rating of 3.4 and a selling price of ₹329 and MRP ₹1,399). The product is sold by IndiaFit with a seller rating of 4.7. Product description: The Ceat Poplar Willow Cricket Bat has been designed to give you power and speed. This cricket bat has been built with a Poplar Willow blade that has thick edges but delivers amazing pick up. With the Sarawak cane handle you can control the bat skillfully and with precision. This top-quality cricket bat is ideal for senior players with intermediate skills.. Highlights include: Age Group 15+ Yrs Blade Made of Poplar Willow Advanced, Training Playing Level Bat Grade: Grade 2 Sport Type: Cricket Weight Range 1.2 kg. [{'source': 'Sports, Books and More'}]
* [SIM=1.860994] The category of th